In [15]:
from dotenv import load_dotenv
import os
load_dotenv("/media/uberdev/ddrv/gitFolders/codeai_fusion/.env")
from dspy import OpenAI

In [2]:
from openinference.instrumentation.dspy import DSPyInstrumentor
# instruments the internal calls in DSPy library
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
# help to get the span of http requests to the APIs
from opentelemetry.sdk import trace as trace_sdk
#processes the data collected from the spans
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.resource import ResourceAttributes
import phoenix as px

INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix
INFO:phoenix.inferences.inferences:Dataset: phoenix_inferences_77bd4f9a-0006-47b3-af62-c2495b5d155b initialized
/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# before executing this cell, ensure phoenix server is running with the postgres database
# will get a warning to start a local server, if not running.
endpoint = "http://127.0.0.1:6006/v1/traces"
client = px.Client(endpoint=endpoint)
# https://docs.arize.com/phoenix/tracing/how-to-tracing/trace-a-deployed-app
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'instrumenting-dspy'
})

INFO:httpx:HTTP Request: GET http://127.0.0.1:6006/v1/traces/arize_phoenix_version "HTTP/1.1 200 OK"


In [11]:
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)

In [12]:
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)

DSPyInstrumentor().instrument(skip_dep_check=True) # here where DSPy is instrumented

In [13]:
# Need to check if the dspy is writing logs
import os
llm = OpenAI(model='gpt-4o-mini',
             api_key=os.environ['OPENAI_API_KEY'],
             max_tokens=2000)

In [16]:
llm("Who is the top player in Fifa")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


['As of my last update in October 2023, the top player in FIFA rankings can vary depending on the specific context, such as the FIFA World Rankings for national teams or the FIFA video game series. \n\nIn terms of the FIFA World Rankings for national teams, players like Lionel Messi and Cristiano Ronaldo have historically been among the most recognized, but the rankings can change frequently based on match performances.\n\nIn the context of the FIFA video game series, the top player ratings can also change annually with new releases. For FIFA 23, for example, Kylian Mbappé and Lionel Messi were among the highest-rated players.\n\nFor the most current information, I recommend checking the latest FIFA World Rankings or the most recent FIFA video game ratings.']